In [16]:
import pandas as pd
from geopy.distance import geodesic
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [17]:
def parse_trackpoint(line):
    # Extract components from the line
    time_utc = line[1:7]  # HHMMSS
    latitude_raw = line[7:15]  # DDMMmmmN
    longitude_raw = line[15:24]  # DDDMMmmmE
    gps_altitude = int(line[25:30])  # GGGG
    pressure_altitude = int(line[30:35])  # LLLL

    # Convert UTC time
    hours = int(time_utc[:2])
    minutes = int(time_utc[2:4])
    seconds = int(time_utc[4:6])

    # Convert latitude to decimal degrees
    latitude_deg = int(latitude_raw[:2])
    latitude_min = float(latitude_raw[2:7]) / 1000
    latitude = latitude_deg + latitude_min / 60
    if latitude_raw[7] == 'S':
        latitude *= -1

    # Convert longitude to decimal degrees
    longitude_deg = int(longitude_raw[:3])
    longitude_min = float(longitude_raw[3:8]) / 1000
    longitude = longitude_deg + longitude_min / 60
    if longitude_raw[8] == 'W':
        longitude *= -1

    return {
        "time": f"{hours:02}:{minutes:02}:{seconds:02}",
        "coordinate": (latitude, longitude),
        "gps_altitude_m": gps_altitude,
        "pressure_altitude_m": pressure_altitude,
    }


In [18]:
file_name = "data/2024-08-03 09_11_21.igc"

with open(file_name, 'r') as file:
    lines = file.readlines()

flight_data = [line for line in lines if line.startswith('B')]
df = pd.DataFrame([parse_trackpoint(line) for line in flight_data])
del lines, flight_data
df.head()

,time,coordinate,gps_altitude_m,pressure_altitude_m
0,09:11:21,"(40.03145, 32.32855)",1142,1147
1,09:11:22,"(40.03145, 32.32855)",1142,1147
2,09:11:23,"(40.03145, 32.32855)",1142,1147
3,09:11:24,"(40.03145, 32.32855)",1142,1147
4,09:11:25,"(40.03143333333333, 32.32855)",1142,1147


In [19]:
for i in range(1, len(df)-1):
    if i % 10 > 0:
        df = df.drop([i])

flight_loginterval = int(df["time"].iloc[1][-2:]) - \
    int(df["time"].iloc[0][-2:])


def calculate_distance(row):
    if row["previus_coordinate"] is None or row["previus_coordinate"] == 0:
        return 0
    return geodesic(row["coordinate"], row["previus_coordinate"]).meters


prev_total_distance = 0


def caluculate_total_distance(row):
    global prev_total_distance
    prev_total_distance += row["distance_m"]
    return prev_total_distance

In [23]:

df["previus_coordinate"] = df["coordinate"].shift(1)
df.loc[0, "previus_coordinate"] = 0
df["distance_m"] = df.apply(calculate_distance, axis=1)
df["total_distance_m"] = df.apply(
    lambda row: caluculate_total_distance(row), axis=1)
df["climb_m"] = df["gps_altitude_m"].diff()
df["climb_rate_m/s"] = df["climb_m"] / flight_loginterval
df["speed_km/s"] = (df["distance_m"]/1000) / (flight_loginterval/3600)
df["distance_from_start_m"] = df.apply(lambda row: geodesic(
    df["coordinate"].iloc[0], row["coordinate"]).meters, axis=1)
df.head()

,time,coordinate,gps_altitude_m,pressure_altitude_m,previus_coordinate,distance_m,total_distance_m,climb_m,climb_rate_m/s,speed_km/s,distance_from_start_m
0,09:11:21,"(40.03145, 32.32855)",1142,1147,0,0.000000,35804.733241,NaN,NaN,0.000000,0.000000
10,09:11:31,"(40.03145, 32.32855)",1142,1149,"(40.03145, 32.32855)",0.000000,35804.733241,0.0,0.0,0.000000,0.000000
20,09:11:41,"(40.03156666666667, 32.328266666666664)",1137,1140,"(40.03145, 32.32855)",27.434746,35832.167987,-5.0,-0.5,9.876508,27.434746
30,09:11:51,"(40.0319, 32.32776666666667)",1147,1149,"(40.03156666666667, 32.328266666666664)",56.490789,35888.658776,10.0,1.0,20.336684,83.468388
40,09:12:01,"(40.0327, 32.328383333333335)",1139,1141,"(40.0319, 32.32776666666667)",103.251459,35991.910235,-8.0,-0.8,37.170525,139.521183


In [21]:
df.to_csv("flight_data.csv", index=False)